In [1]:
import json
import glob #Search files
import re
import os #Path things
import pickle

import numpy as np
from scipy.spatial.transform import Rotation

In [2]:
dataset_path = os.environ["USERPROFILE"]+"\\AppData\\LocalLow\\DefaultCompany\\IA904-3D_Pose\\solo"
dataset_path = "I:\\.shortcut-targets-by-id\\1S6q0nt4z5LYa-5VkpC8qxag2b_jjO6e9\\IA904\\Dataset"

In [3]:
search_pattern = dataset_path+"\\**\\*frame_data.json"
frame_data_paths = glob.glob(search_pattern, recursive=True)

In [4]:
scenario_regex = re.compile("\\\\Scenario[0-9]\\\\")

In [5]:
keypoint_type_3d = "type.unity.com/unity.solo.Keypoint3dAnnotation"
keypoint_type_2d = "type.unity.com/unity.solo.KeypointAnnotation"

camera_id_left = "camera_left"
camera_id_right = "camera_right"
camera_id_center = "camera_center"

camera_ids = [camera_id_left, camera_id_right, camera_id_center]

In [6]:
labels = ['hip',
 'leg_left',
 'knee_left',
 'accessories_ankle_left',
 'ankle_left',
 'foot_left',
 'toes_left',
 'lower_leg_left',
 'upper_leg_left',
 'leg_right',
 'knee_right',
 'accessories_ankle_right',
 'ankle_right',
 'foot_right',
 'toes_right',
 'lower_leg_right',
 'upper_leg_right',
 'spine_01',
 'spine_02',
 'spine_03',
 'clavicle_left',
 'shoulder_left',
 'elbow_left',
 'accessories_wrist_left',
 'lower_arm_left',
 'wrist_left',
 'index_01_left',
 'index_02_left',
 'index_03_left',
 'index_04_left',
 'middle_01_left',
 'middle_02_left',
 'middle_03_left',
 'middle_04_left',
 'palm_left',
 'pinky_01_left',
 'pinky_02_left',
 'pinky_03_left',
 'pinky_04_left',
 'ring_01_left',
 'ring_02_left',
 'ring_0',
 'ring_03_left',
 'ring_04_left',
 'thumb_01_left',
 'thumb_02_left',
 'thumb_03_left',
 'thumb_04_left',
 'upper_arm_left',
 'clavicle_right',
 'shoulder_right',
 'elbow_right',
 'accessories_wrist_right',
 'lower_arm_right',
 'wrist_right',
 'index_01_right',
 'index_02_right',
 'index_03_right',
 'index_04_right',
 'middle_01_right',
 'middle_02_right',
 'middle_03_right',
 'middle_04_right',
 'palm_right',
 'pinky_01_right',
 'pinky_02_right',
 'pinky_03_right',
 'pinky_04_right',
 'ring_01_right',
 'ring_02_right',
 'ring_03_right',
 'ring_04_right',
 'thumb_01_right',
 'thumb_02_right',
 'thumb_03_right',
 'thumb_04_right',
 'upper_arm_right',
 'neck',
 'head',
 'accessories_radix_nose',
 'eye_left',
 'eye_right',
 'head_end',
 'jaw',
 'jaw_end',
 'ear_left',
 'ear_right',
 'nose',
 'hip_left',
 'hip_right']

label_to_index = {labels[i]:i for i in range(len(labels))}

In [7]:
dataset_index = []

for file_path in frame_data_paths:
    with open(file_path) as file:
        frame_data = json.load(file)

    scenario = int(scenario_regex.findall(file_path)[0].split("\\Scenario")[1].split("\\")[0])
    sequence = frame_data["sequence"]
    step = frame_data["step"]

    has_keypoints = False
    keypoints_3d = np.full((len(labels), 3), np.nan, np.float32)

    transformation = np.zeros((4,4), np.float32)
    transformation[3,3] = 1

    for capture in frame_data["captures"]:

        if capture["id"] == camera_id_center:
            rotation = capture["rotation"]
            rotation = Rotation.from_quat(rotation)
            rotation = rotation.as_matrix()

            transformation[:3,:3] = rotation

            position = capture["position"]

            transformation[:3,3] = position


        for annotation in capture["annotations"]:
            if annotation["@type"] == keypoint_type_3d:
                has_keypoints = True

                for keypoint in annotation["keypoints"][0]["keypoints"]:
                    label = keypoint["label"]
                    index = label_to_index[label]
                    keypoints_3d[index] = keypoint["location"]
    
    #World to camera
    transformation_w2c = np.linalg.inv(transformation)

    if not has_keypoints:
        continue
    
    position = np.empty(4)
    position[3] = 1
    for i in range(len(labels)):
        position[:3] = keypoints_3d[i]
        
        position_camera = transformation_w2c@position

        keypoints_3d[i] = position_camera[:3]

    data = {"scenario":scenario, 
            "sequence":sequence, 
            "step":step, 
            "keypoints_3d":keypoints_3d}

    dataset_index.append(data)



In [8]:
index_path = os.path.join(dataset_path, "index.pickle")
file = open(index_path, "wb")
pickle.dump(dataset_index, file)
file.close()

In [9]:
len(dataset_index)

6400